In [1]:
!pip install segmentation-models --quiet
import tensorflow as tf

import keras
import keras.backend as K
import keras.callbacks as cbs
import cv2
import pandas as pd
import numpy as np
import os, shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, train_test_split

In [2]:
train=pd.read_csv('../input/cloud-images-resized/train_384x576.csv')
train['Image']=train['Image_Label'].map(lambda x: x.split("_")[0])
train['Label']=train['Image_Label'].map(lambda x: x.split("_")[1])
train2=pd.DataFrame({'Image':train['Image'][::4]})
train2['e1']=train['EncodedPixels'][::4].values
train2['e2']=train['EncodedPixels'][1::4].values
train2['e3']=train['EncodedPixels'][2::4].values
train2['e4']=train['EncodedPixels'][3::4].values
train2.set_index('Image', drop=True, inplace=True)
train2=train2.fillna('')
train2[['d1','d2','d3','d4']]=(train2[['e1','e2','e3','e4']]!='').astype(int)
train2.head()

,e1,e2,e3,e4,d1,d2,d3,d4
Image,,,,,,,,
0011165.jpg,20056 257 20440 257 20824 257 21208 257 21592 ...,102245 275 102629 275 103013 275 103397 275 10...,,,1,1,0,0
002be4f.jpg,17668 241 18051 242 18435 242 18819 242 19203 ...,100850 142 101234 142 101618 142 102002 142 10...,,5074 96 5458 96 5842 96 6226 96 6610 96 6994 9...,1,1,0,1
0031ae9.jpg,579 190 963 190 1347 190 1731 190 2115 190 249...,178 193 562 193 946 193 1330 193 1714 193 2098...,,49583 107 49967 107 50351 107 50735 107 51119 ...,1,1,0,1
0035239.jpg,,7684 127 8068 127 8452 127 8836 127 9220 127 9...,5267 104 5651 104 6035 104 6419 104 6803 104 7...,,0,1,1,0
003994e.jpg,178332 27 178715 28 179089 2 179092 35 179470 ...,,26639 114 27023 114 27407 114 27791 114 28175 ...,1924 133 2308 134 2692 134 3076 134 3460 134 3...,1,0,1,1


In [3]:
def rle2mask(rle,shrink=1,shape=(2100,1400)):
    rle=rle.split()
    starts,lengths=[np.asarray(x,dtype=int) for (x) in(rle[::2],rle[1::2]) ]
    starts-=1
    ends=starts+lengths
    mask=np.zeros(shape[0]*shape[1],dtype='uint8')
    
    for lo,hi in zip(starts,ends):
        mask[lo:hi]=1
        
    return mask.reshape(shape).T[::shrink,::shrink]
        

In [4]:
import albumentations as albu
%env SM_FRAMEWORK=tf.keras
import segmentation_models as sm

env: SM_FRAMEWORK=tf.keras
Segmentation Models: using `tf.keras` framework.


In [5]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self,list_ids,shrink1=1,shrink2=1,dim=(576,384),height= 352, width=544,
                path='../input/cloud-images-resized/train_images_384x576/',
                 flips=False, augment=False,batch_size=32,scale=1/128,sub=1,mode='train',shuffle=False):
        self.shrink1=shrink1
        self.shrink2=shrink2
        self.mode=mode
        self.path=path
        self.dim=dim
        self.augment=augment
        self.height=height
        self.width=width
        self.list_ids=list_ids
        self.batch_size=batch_size
        self.flips=flips
        self.scale=scale
        self.sub=sub
        self.shuffle=shuffle
        self.on_epoch_end()
        
    def __len__(self):
        ct= int(np.floor(len(self.list_ids)/(self.batch_size)))
        if len(self.list_ids)>ct*self.batch_size:
            ct+=1
        return ct
    
    def __getitem__(self,index):
        indexes=self.indexes[index*self.batch_size : (index+1)*self.batch_size]
        X,msk=self.__generate_data(indexes)
#         print(X[1])
        if self.augment : X,msk=self.__batch_aug(X,msk)
        if (self.mode=='train')|(self.mode=='val'):
            return X,msk
        else : return X
        
    def __generate_data(self,indexes):
        
        lnn= len(indexes) ;ex=self.shrink1 ;ax =self.shrink2
        X = np.empty((lnn,self.height, self.width,3),dtype=np.float32)
        msk=np.empty((lnn,self.height,self.width,4),dtype=np.float32)
        
        for i in range(lnn):
            img=cv2.imread(self.path+self.list_ids[indexes[i]])
            hflips=False ; vflips=False
            if(self.flips):
                if np.random.uniform(0,1)>0.5 : hflips=True
                if np.random.uniform(0,1)>0.5 : vflips=True
            if hflips:
                img=cv2.flip(img,1)
            if vflips:
                img=cv2.flip(img,0)
            
            ## shake augment
            a=np.random.randint(0,self.dim[0]//ax//ex-self.width/ax+1)
            
            b=np.random.randint(0,self.dim[1]//ax//ex-self.height/ax+1)
            
            if (self.mode=='predict'):
                a=(self.dim[0]//ex//ax-self.width//ax)//2
                b=(self.dim[1]//ex//ax-self.height//ax)//2
                
            img=img[b*ax:self.height+b*ax,a*ax:self.width+a*ax]
            X[i]=img*self.scale - self.sub
            
            
            if (self.mode!='predict'):
                
                for j in range(1,5):
                    rle=train2.loc[self.list_ids[indexes[i]],'e'+str(j)]
                    mask=rle2mask(rle,shrink=ex*ax,shape=self.dim)
                    if hflips:
                        mask=np.flip(mask,axis=1)
                    if vflips:
                        mask=np.flip(mask,axis=0)
                    msk[i,:,:,j-1]=mask[b:self.height//ax+b,a:self.width//ax+a]
                    
                    
        return X,msk
    
    def on_epoch_end(self):
        self.indexes=np.arange(int(len(self.list_ids)))
        if self.shuffle: np.random.shuffle(self.indexes)
            
    def __random_transform(self, img,mask):
        composition=albu.Compose([
        albu.ShiftScaleRotate(scale_limit=0.1,rotate_limit=30,p=0.5)
        ])
        
        composed=composition(image=img,mask=mask)
#         print(composed['image'])
        return composed['image'],composed['mask']
        
    def __batch_aug(self,img_batch,mask_batch):
        for i in  range(img_batch.shape[0]):
            img_batch[i],mask_batch[i]=self.__random_transform(img_batch[i],mask_batch[i])
#         print(img_batch[1])
        return img_batch, mask_batch
                
                
                
            
                

In [6]:
# train_gen=DataGenerator(train2.index,augment=True,shuffle=True)
# X,msk=train_gen.__getitem__(3)
# print(X.shape,msk.shape)
# for i in range(5):
#     fig,ax=plt.subplots(figsize=(20,20))
#     ax.imshow(X[i])
#     ax.imshow(msk[i])
#     plt.show()

In [7]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [ ]:
filters = [256, 128, 64, 32, 16]
REDUCTION = 0; RED = 2**REDUCTION
filters = filters[:5-REDUCTION]

BATCH_SIZE = 16
jaccard_loss = sm.losses.JaccardLoss() 

skf = KFold(n_splits=3, shuffle=True)#, random_state=RAND)
# for k, (idxT0, idxV0) in enumerate( skf.split(train2) ):
train_idx,val_idx=train_test_split(train2.index,test_size=0.1)
# train_idx = train2.index[idxT0]
# val_idx = train2.index[idxV0]

# if k==0: idx_oof_0 = val_idx.copy()
# elif k==1: idx_oof_1 = val_idx.copy()
# elif k==2: idx_oof_2 = val_idx.copy()

# print('#'*20)
# print('### Fold',k,'###')
# print('#'*20)

#     if not DO_TRAIN: continue

train_generator = DataGenerator(
    train_idx, flips=True, augment=True, shuffle=True, shrink2=RED, batch_size=BATCH_SIZE,
)

val_generator = DataGenerator(
    val_idx, shrink2=RED, batch_size=BATCH_SIZE
)

opt =keras.optimizers.Adam(learning_rate=0.001)# AdamAccumulate(lr=0.001, accum_iters=8)
model2 = sm.Unet(
    'efficientnetb2', 
    classes=4,
    encoder_weights='imagenet',
    decoder_filters = filters,
    input_shape=(None, None, 3),
    activation='sigmoid'
)
model2.compile(optimizer=opt, loss=jaccard_loss, metrics=[dice_coef])#,kaggle_dice,kaggle_acc])

checkpoint = cbs.ModelCheckpoint('model_'+str(1)+'.h5', save_best_only=True)
es = cbs.EarlyStopping(monitor='val_dice_coef', min_delta=0.001, patience=7, verbose=1, mode='max')
rlr = cbs.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=2, verbose=1, mode='max', min_delta=0.001)

history = model2.fit_generator(
     train_generator,
     validation_data=val_generator,
     callbacks=[rlr, es, checkpoint],
     epochs=50,
     verbose=1, workers=2
)

31940608/31936256 [==============================] - 1s 0us/step
Epoch 1/50
312/312 [==============================] - 331s 1s/step - loss: 0.6713 - dice_coef: 0.4821 - val_loss: 0.6733 - val_dice_coef: 0.4919
Epoch 2/50
312/312 [==============================] - 326s 1s/step - loss: 0.6089 - dice_coef: 0.5577 - val_loss: 0.6162 - val_dice_coef: 0.5468
Epoch 3/50
312/312 [==============================] - 324s 1s/step - loss: 0.5993 - dice_coef: 0.5681 - val_loss: 0.6826 - val_dice_coef: 0.4988
Epoch 4/50
312/312 [==============================] - ETA: 0s - loss: 0.5898 - dice_coef: 0.5769
Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
312/312 [==============================] - 324s 1s/step - loss: 0.5898 - dice_coef: 0.5769 - val_loss: 0.6465 - val_dice_coef: 0.5179
Epoch 5/50
312/312 [==============================] - 324s 1s/step - loss: 0.5760 - dice_coef: 0.5915 - val_loss: 0.5891 - val_dice_coef: 0.5931
Epoch 6/50
312/312 [========================

In [ ]:
pp

In [ ]:
BATCH_SIZE = 16

model1 = sm.Unet(
    'efficientnetb1', 
    classes=4,
    encoder_weights='imagenet',
    decoder_filters = filters,
    input_shape=(None, None, 3),
    activation='sigmoid'
)
model2.compile(optimizer=opt, loss=jaccard_loss, metrics=[dice_coef])#,kaggle_dice,kaggle_acc])

checkpoint = cbs.ModelCheckpoint('model_'+str(2)+'.h5', save_best_only=True)
es = cbs.EarlyStopping(monitor='val_dice_coef', min_delta=0.001, patience=7, verbose=1, mode='max')
rlr = cbs.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=2, verbose=1, mode='max', min_delta=0.001)

history = model2.fit_generator(
     train_generator,
     validation_data=val_generator,
     callbacks=[rlr, es, checkpoint],
     epochs=50,
     verbose=1, workers=2
)

In [ ]:
BATCH_SIZE = 10


train_generator = DataGenerator(
    train_idx, flips=True, augment=True, shuffle=True, shrink2=RED, batch_size=BATCH_SIZE,
)

val_generator = DataGenerator(
    val_idx, shrink2=RED, batch_size=BATCH_SIZE
)

model3 = sm.Unet(
    'efficientnetb3', 
    classes=4,
    encoder_weights='imagenet',
    decoder_filters = filters,
    input_shape=(None, None, 3),
    activation='sigmoid'
)
model3.compile(optimizer=opt, loss=jaccard_loss, metrics=[dice_coef])#,kaggle_dice,kaggle_acc])

checkpoint = cbs.ModelCheckpoint('model_'+str(3)+'.h5', save_best_only=True)
es = cbs.EarlyStopping(monitor='val_dice_coef', min_delta=0.001, patience=7, verbose=1, mode='max')
rlr = cbs.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=2, verbose=1, mode='max', min_delta=0.001)

history = model3.fit_generator(
     train_generator,
     validation_data=val_generator,
     callbacks=[rlr, es, checkpoint],
     epochs=50,
     verbose=1, workers=2
)

In [ ]:
BATCH_SIZE = 10


model4 = sm.Unet(
    'efficientnetb4', 
    classes=4,
    encoder_weights='imagenet',
    decoder_filters = filters,
    input_shape=(None, None, 3),
    activation='sigmoid'
)
model2.compile(optimizer=opt, loss=jaccard_loss, metrics=[dice_coef])#,kaggle_dice,kaggle_acc])

checkpoint = cbs.ModelCheckpoint('model_'+str(4)+'.h5', save_best_only=True)
es = cbs.EarlyStopping(monitor='val_dice_coef', min_delta=0.001, patience=7, verbose=1, mode='max')
rlr = cbs.ReduceLROnPlateau(monitor='val_dice_coef', factor=0.5, patience=2, verbose=1, mode='max', min_delta=0.001)

history = model2.fit_generator(
     train_generator,
     validation_data=val_generator,
     callbacks=[rlr, es, checkpoint],
     epochs=50,
     verbose=1, workers=2
)

In [ ]:
sub = pd.read_csv('../input/understanding_cloud_organization/sample_submission.csv')
sub['Image'] = sub['Image_Label'].map(lambda x: x.split('.')[0])
sub['Label'] = sub['Image_Label'].map(lambda x: x.split('_')[1])
# LOAD TEST CLASSIFIER PREDICTIONS
sub['p'] = pd.read_csv('../input/cloud-classifiers/pred_cls.csv').p.values
sub['p'] += np.load('../input/cloud-classifiers/pred_cls0.npy').reshape((-1)) * 0.5
sub['p'] += pd.read_csv('../input/cloud-classifiers/pred_cls3.csv').p.values * 3.0
sub['p'] += np.load('/kaggle/input/cloud-classifiers/pred_cls4b.npy') * 0.6
sub['p'] /= 5.1

In [ ]:
USE_TTA=True
DO_TEST=True

In [ ]:
print('Computing masks for',len(sub)//4,'test images with 3 models'); sub.EncodedPixels = ''
PTH = '../input/cloud-images-resized/test_images_384x576/'; bs = 4
if USE_TTA: bs=1
test_gen = DataGenerator(sub.Image[::4].values+'.jpg', width=576, height=384, batch_size=bs, mode='predict',path=PTH)

sz = 20000.*(576/525)*(384/350)/RED/RED

pixt = [0.5,0.5,0.5,0.35] #; pixt = [0.4,0.4,0.4,0.4]
szt = [25000., 20000., 22500., 15000.] #; szt = [20000., 20000., 20000., 20000.]
for k in range(len(szt)): szt[k] = szt[k]*(576./525.)*(384./350.)/RED/RED

if DO_TEST:
    for b,batch in enumerate(test_gen):
        btc = model1.predict_on_batch(batch)
        btc += model2.predict_on_batch(batch)
        btc += model3.predict_on_batch(batch)
        btc += model4.predict_on_batch(batch)
        btc /= 4.0

        for j in range(btc.shape[0]):
            for i in range(btc.shape[-1]):
                mask = (btc[j,:,:,i]>pixt[i]).astype(int); rle = ''
                if np.sum(mask)>szt[i]: rle = mask2rleXXX( mask ,shape=(576//RED,384//RED))
                sub.iloc[4*(bs*b+j)+i,1] = rle
        if b%(100//bs)==0: print(b*bs,', ',end='')
#         t = np.round( (time.time() - kernel_start)/60,1 )
#         if t > LIMIT*60:
#             print('#### EXCEEDED TIME LIMIT. STOPPING NOW ####')
#             break

    sub[['Image_Label','EncodedPixels']].to_csv('sub_seg.csv',index=False)
    sub.loc[(sub.p<0.5)&(sub.Label=='Fish'),'EncodedPixels'] = ''
    sub.loc[(sub.p<0.3)&(sub.Label=='Flower'),'EncodedPixels'] = ''
    sub.loc[(sub.p<0.5)&(sub.Label=='Gravel'),'EncodedPixels'] = ''
    sub.loc[(sub.p<0.5)&(sub.Label=='Sugar'),'EncodedPixels'] = ''
    sub[['Image_Label','EncodedPixels']].to_csv('submission.csv',index=False)

sub.head(10)